## Grid Search for HyperParameter Tuning

This part is used to show the process of hyper parameter tuning for ridge regression using grid search.

### Data loading and feature selection

The first part is data loading and feature selection, which is the same as in the main.ipynb file. We read data, separate them into three groups according to jet_num and then drop features which are not defined or are highly correlated.

In [1]:
# import basic packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# import self-defined modules
from implementations import *
from tools_xia import *
from helpers_xia import *

# just to ingore warning
import warnings
warnings.filterwarnings('ignore')

# import pandas as pd

In [2]:
data_path_tr = 'data/train.csv'
yb_tr, data_tr, idx_tr, labels = load_csv_data(data_path_tr, sub_sample=False)

# prepare the data using self-defined DataFrame class
labels_dataframe = ['Prediction'] + labels
data_tr_dataframe = np.concatenate((yb_tr.reshape([-1,1]), data_tr), axis=1)
dataframe_tr = DataFrame(data_tr_dataframe, idx_tr.tolist(), labels_dataframe)

In [3]:
DER_mode_s =119.89
DER_mode_b =96.819# 多个值有相同个数，所以于原本不同

# replace -999（undefined values of 'DER_mass_MMC'） with mode respectively
temp = dataframe_tr.loc(dataframe_tr['DER_mass_MMC']==-999)
temp = temp.loc(temp['Prediction']==1)
dataframe_tr.loc(temp.index)['DER_mass_MMC'] = DER_mode_s

temp = dataframe_tr.loc(dataframe_tr['DER_mass_MMC']==-999)
temp = temp.loc(temp['Prediction']==-1)
dataframe_tr.loc(temp.index)['DER_mass_MMC'] = DER_mode_b

In [4]:
feature_dorp_phi = ['PRI_jet_leading_phi',
                    'PRI_jet_subleading_phi',
                    'PRI_lep_phi',
                    'PRI_met_phi',
                    'PRI_tau_phi',]
dataframe_tr = dataframe_tr.drop(feature_dorp_phi)

In [5]:
def group_features_by_jet(dataframe):
    return {  
        0: dataframe.loc( dataframe['PRI_jet_num'] == 0).copy(),
        1: dataframe.loc( dataframe['PRI_jet_num'] == 1).copy(),
        2: dataframe.loc((dataframe['PRI_jet_num'] == 2) | (dataframe['PRI_jet_num'] == 3)).copy(),
    }
dataframe_tr_grp = group_features_by_jet(dataframe_tr)
dataframe_tr_grp[0].values

array([[  -1.   ,  143.905,   81.417, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,  175.864,   16.915, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,  105.594,   50.559, ..., -999.   , -999.   ,    0.   ],
       ...,
       [  -1.   ,   96.819,   58.179, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,   94.951,   19.362, ..., -999.   , -999.   ,    0.   ],
       [  -1.   ,   96.819,   72.756, ..., -999.   , -999.   ,    0.   ]])

In [6]:
# calculate missing rate 
def missing_rate(dataframe, miss_number):
    features = dataframe.drop(['Prediction'])
    missing_rate = []
    for feature in features.labels:
        data = features.loc(features[feature] ==  miss_number)
        if data.values.size != 0:
            missing_rate.append(data[feature].size / features[feature].size)
        else:
            missing_rate.append(0)
    missing_rate = np.array(missing_rate).reshape([1, -1]).squeeze()
    missing_rate_labels = dataframe.labels[1:]
    return DataFrame(missing_rate, [0], missing_rate_labels)

# get features with undefined values
dataframe_tr_feature_undefined = []
for i in range(len(dataframe_tr_grp)):
    dataframe_tr_miss = missing_rate(dataframe_tr_grp[i], -999)
    dataframe_tr_feature_undefined.append(np.array(dataframe_tr_miss.labels)[dataframe_tr_miss.values == 1].tolist())
dataframe_tr_feature_undefined

[['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta'],
 ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta'],
 []]

In [7]:
# drop features with undefined values in each group
dataframe_tr_grp[0] = dataframe_tr_grp[0].drop(dataframe_tr_feature_undefined[0])
dataframe_tr_grp[1] = dataframe_tr_grp[1].drop(dataframe_tr_feature_undefined[1])
# group2 have no feature with undefined values
dataframe_tr_grp[0].values

array([[ -1.   , 143.905,  81.417, ...,  86.062,   0.   ,   0.   ],
       [ -1.   , 175.864,  16.915, ...,  53.131,   0.   ,   0.   ],
       [ -1.   , 105.594,  50.559, ..., 129.804,   0.   ,   0.   ],
       ...,
       [ -1.   ,  96.819,  58.179, ...,  80.408,   0.   ,   0.   ],
       [ -1.   ,  94.951,  19.362, ..., 112.718,   0.   ,   0.   ],
       [ -1.   ,  96.819,  72.756, ...,  99.405,   0.   ,   0.   ]])

In [8]:
dataframe_tr_feature_zero = []
for i in range(len(dataframe_tr_grp)):
    dataframe_tr_miss = missing_rate(dataframe_tr_grp[i], 0)
    dataframe_tr_feature_zero.append(np.array(dataframe_tr_miss.labels)[dataframe_tr_miss.values == 1].tolist())
dataframe_tr_feature_zero

[['PRI_jet_num', 'PRI_jet_all_pt'], [], []]

In [9]:
# drop features with undefined values in each group
# Because only the first group has all zeros column, here we only 
dataframe_tr_grp[0] = dataframe_tr_grp[0].drop(dataframe_tr_feature_zero[0])
dataframe_tr_grp[0].values

array([[ -1.   , 143.905,  81.417, ...,  -0.522,  31.082,  86.062],
       [ -1.   , 175.864,  16.915, ...,   0.798,   2.723,  53.131],
       [ -1.   , 105.594,  50.559, ...,   0.21 ,  37.791, 129.804],
       ...,
       [ -1.   ,  96.819,  58.179, ...,   0.308,  46.737,  80.408],
       [ -1.   ,  94.951,  19.362, ...,  -0.874,  12.15 , 112.718],
       [ -1.   ,  96.819,  72.756, ...,   1.49 ,  40.729,  99.405]])

In [10]:
# drop feature 'PRI_jet_num' which is already used for grouping
dataframe_tr_grp[1] = dataframe_tr_grp[1].drop('PRI_jet_num')
dataframe_tr_grp[2] = dataframe_tr_grp[2].drop('PRI_jet_num')

In [11]:
feature_dorp_corr0 = list(['DER_mass_MMC','DER_pt_tot','PRI_tau_pt','PRI_lep_pt'])
feature_dorp_corr1 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])
feature_dorp_corr2 = list(['DER_mass_MMC','DER_pt_h', 'DER_sum_pt', 'PRI_met_sumet','PRI_jet_all_pt','PRI_lep_pt'])

dataframe_tr_grp[0] = dataframe_tr_grp[0].drop(feature_dorp_corr0)
dataframe_tr_grp[1] = dataframe_tr_grp[1].drop(feature_dorp_corr1)
dataframe_tr_grp[2] = dataframe_tr_grp[2].drop(feature_dorp_corr2)

### Starting Grid Search 

After loading data and selecting features, now we can start hyper parameter tuning. Here we define functions `grid_serach` and `ridge_grid_search` to implement grid search and function `get_best_parameters` to return best parameters based on test set accuracy.

In [12]:
def get_best_parameters(para1,para2, acc):
    """
    Get the best degree and lambda from the result of grid search.
    
    Inputs:
        para1,para2:
        Range of parameters in numpy.array.
        
        acc:
        Accuracy matrix saving the accuracy of each 
    
    """
    max_row, max_col = np.unravel_index(np.argmax(acc), acc.shape)
    return acc[max_row, max_col], para1[max_row], para2[max_col]

In [13]:
def grid_search(y, tx, para1, para2):
    """
    Function implements grid search.
    
    Inputs:
        y,tx:
        Label and data for model training.
        
        para1,para2:
        Range of parameters in numpy.array.
    
    Result:
        acc_tr,acc_te:
        Average accuracy of 10-fold cross validation for training set and test set.
    """
    acc_tr = np.zeros((len(para1), len(para2)))
    acc_te = np.zeros((len(para1), len(para2)))
    tx = log_process(tx)
    # k_indices = build_k_indices(y, k_fold=3, seed=1)
    for i in range(0,len(para1)):
        for j in range(0,len(para2)):
            tx,_,_ = standardize(tx)
            tx_poly = build_poly(tx, para1[i])
            # tx_poly,_,_ = standardize(tx_poly)
            lambda_ = para2[j]
            #_,acc_tr[i][j],acc_te[i][j] = cross_validation(y, tx_poly, k_indices, k=0, regression_method=ridge_regression, lambda_=lambda_)
            _,acc_tr[i][j],acc_te[i][j] = cv_loop(y=y, x=tx_poly, k_fold=3, seed=10, regression_method=ridge_regression, lambda_=lambda_)
    return acc_tr,acc_te

In [14]:
def ridge_grid_search(degrees,lambdas):
    """
    The function is used for hyper parameter tuning for ridge regression using grid search.
    
    Inputs:
        degrees: 
        Numpy.array. Degrees range for grid search.
        
        lambdas: 
        Numpy.array. Lambdas range for grid search.
        
    Return:
        The average accuracy of 10-fold cross validation of training set and test set.
    """
    acc_tr = []
    acc_te = []
    for i in range(len(dataframe_tr_grp)):
        tx = dataframe_tr_grp[i].drop('Prediction').values
        # tx = feature_tr_grp[i].drop(columns = ['Id','Prediction']).values
        y = (dataframe_tr_grp[i])['Prediction']
        # y = feature_tr_grp[i].Prediction.replace(['s','b'],[1,-1]).values
        acc_tr_tmp,acc_te_tmp = grid_search(y,tx,degrees,lambdas)
        acc_tr.append(acc_tr_tmp)
        acc_te.append(acc_te_tmp)
    return acc_tr,acc_te

We set the degrees 

In [ ]:
#degrees = np.arange(11)+1
degrees = [1,5,9,11,13,15]
lambdas = [0.001,0.01,0.1]
acc_tr,acc_te = ridge_grid_search(degrees,lambdas)

In [ ]:
def plot_gridsearch(acc_tr,acc_te,degrees,lambdas):
    """
    This function is used to plot the result of grid search on accuracy over different values of hyper parameters.
    
    Inputs:
        acc_tr,acc_te: 
        The average accuracy of 10-fold cross validation of training set and test set.
        
        degrees,lambdas:
        Numpy.array. Degrees and lambdas range for grid search.
    """
    fig,ax = plt.subplots(3,3,figsize = (30,30))
    ax = ax.ravel()

    for j in range(len(acc_tr)):
        for i in range(acc_tr[0].shape[1]):
            ax[3*j+i].plot(degrees, acc_tr[j][:,i], color='r', marker='s', label="Train Accuray" )
            ax[3*j+i].plot(degrees, acc_te[j][:,i], color='b', marker='v', label="Test Accuracy" )
            ax[3*j+i].plot(degrees, (acc_te[j][:,i]+acc_tr[j][:,i])/2, color='g', marker='^', label="Average Accuracy" )
            ax[3*j+i].set_title('Lambda = '+str(lambdas[i]))
            ax[3*j+i].set_xlabel('Degree',size=15)
            ax[3*j+i].set_ylabel('Accuracy',size=15)
            ax[3*j+i].legend()
        acc,degree,lambda_ = get_best_parameters(degrees,lambdas, acc_te[j])
        print("For group jet_num = {j}, best test accuracy={acc}, poly degree={degree}, lambda={lambda_}".format(j = j,acc=acc, degree=degree, lambda_ = lambda_))
    fig.suptitle('Grid Search Result over Polynomial Degree and Lambda',size=30,y=0.93)
    fig.text(0.5, 0.9, 'Group jet_num = 0', ha='center', va='center',size = 25)
    fig.text(0.5, 0.63, 'Group jet_num = 1', ha='center', va='center',size = 25)
    fig.text(0.5, 0.35, 'Group jet_num = 2,3', ha='center', va='center',size = 25)
    fig.subplots_adjust(hspace=.5)

In [ ]:
plot_gridsearch(acc_tr,acc_te,degrees,lambdas)